In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import logging

module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pathlib import Path

import torch
import torch.utils.data
from torchvision import transforms, datasets
import numpy as np
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
import matplotlib.pyplot as plt

import pvi
from pvi.models import ClassificationBNNLocalRepam
from pvi.clients import Client
from pvi.servers import SequentialServer, ContinualLearningServer
from pvi.distributions import MeanFieldGaussianDistribution, MeanFieldGaussianFactor
from pvi.utils.training_utils import EarlyStopping

In [2]:
data_dir = Path("/Users/terjemildner/Desktop/data")
cache_dir = Path("/Users/terjemildner/Desktop/PVI-main/experiments/fekmnist")

## Define various functions for splitting data and recording performance metrics.

In [3]:
# Data splitting functions.
def homogeneous_split(data, num_clients=100, seed=42):
    # Set numpy's random seed.
    np.random.seed(seed)
    
    perm = np.random.permutation(len(data["x"]))
    client_data = []
    for i in range(num_clients):
        client_idx = perm[i::num_clients]
        client_data.append({"x": data["x"][client_idx], "y": data["y"][client_idx]})
    
    return client_data

def heterogeneous_split(data, num_clients=100):
    # Set numpy's random seed.
    listed = np.argsort(data["y"])

    sorted_x = data["x"][listed[::1]]
    sorted_y = data["y"][listed[::1]]
    
    l = len(data["y"])
    client_size = int(np.ceil(l/num_clients))
    
    client_data = []
    
    for i in range(num_clients):        
        client_data.append({"x": sorted_x[i*client_size:(i+1)*client_size], 
                            "y": sorted_y[i*client_size:(i+1)*client_size]})
    
    return client_data

# Performance metric function. Why is the batch size 512 when the split gives 600? => Mini batches?
def performance_metrics(client, data, batch_size=512):
    dataset = torch.utils.data.TensorDataset(data["x"], data["y"])
    loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
    
    device = client.config["device"]
    
    if device == "cuda":
        loader.pin_memory = True
        
    preds, mlls = [], []
    for (x_batch, y_batch) in loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        pp = client.model_predict(x_batch)
        preds.append(pp.component_distribution.probs.mean(1).cpu())
        mlls.append(pp.log_prob(y_batch).cpu())
        
    mll = torch.cat(mlls).mean()
    preds = torch.cat(preds)
    acc = sum(torch.argmax(preds, dim=-1) == loader.dataset.tensors[1]) / len(
        loader.dataset.tensors[1]
    )
    
    return {"mll": mll, "acc": acc}

## Data preprocessing

In [4]:
transform_train = transforms.Compose([transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToTensor()])

train_set = datasets.KMNIST(root=data_dir, train=True, download=True, transform=transform_train)
test_set = datasets.KMNIST(root=data_dir, train=False, download=True, transform=transform_test)

train_data = {
    "x": ((train_set.data - 0) / 255).reshape(-1, 28 * 28),
    "y": train_set.targets,
}

test_data = {
    "x": ((test_set.data - 0) / 255).reshape(-1, 28 * 28),
    "y": test_set.targets,
}

In [5]:
# Get client splits.
client_data = homogeneous_split(train_data, 100, seed=42)


In [6]:
client_data_2 = heterogeneous_split(train_data, 100)

## Define configuration for server and clients

In [7]:
model_config = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 10,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr": 25e-4},
    "sigma_optimiser_params": {"lr": 25e-4},
    "early_stopping": EarlyStopping(100, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 256,
    "epochs": 4000,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
}

server_config = {
    **client_config,
    #100
    "max_iterations": 1,
}

## Set up model etc.

In [8]:
device = server_config["device"]

model = ClassificationBNNLocalRepam(config=model_config)

# Initial parameters.
init_q_std_params = {
    "loc": torch.zeros(size=(model.num_parameters,)).to(device).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model.num_parameters,)).to(device) 
    * client_config["init_var"] ** 0.5,
}

prior_std_params = {
    "loc": torch.zeros(size=(model.num_parameters,)).to(device),
    "scale": model_config["prior_var"] ** 0.5 
    * torch.ones(size=(model.num_parameters,)).to(device),
}

init_factor_nat_params = {
    "np1": torch.zeros(model.num_parameters).to(device),
    "np2": torch.zeros(model.num_parameters).to(device),
}

p = MeanFieldGaussianDistribution(
    std_params=prior_std_params, is_trainable=False
)
init_q = MeanFieldGaussianDistribution(
    std_params=init_q_std_params, is_trainable=False
)

clients = []
size_training = 0
for i in range(100):
    data_i = client_data[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params)
    clients.append(
        Client(
            data=data_i,
            model=model,
            t=t_i,
            config=client_config,
            val_data=test_data
        )
    )    
server = SequentialServer(model=model, p=p, clients=clients, config=server_config, init_q=init_q, data=train_data, val_data=test_data)

## Run PVI! Online

In [15]:
i = 0
while not server.should_stop():
    server.tick()

    # Obtain performance metrics.
    metrics = server.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Iterations: 0.
Time taken: 409.521.
Test mll: -0.755. Test acc: 0.771.
Train mll: -0.297. Train acc: 0.911.



# Run PVI! Online with 2NN

In [16]:
model_config2 = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 10,
    "num_layers": 2,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config2 = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr": 25e-4},
    "sigma_optimiser_params": {"lr": 25e-4},
    "early_stopping": EarlyStopping(100, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 256,
    "epochs": 4000,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
}

server_config2 = {
    **client_config,
    #100
    "max_iterations": 1,
}

In [17]:
device2 = server_config2["device"]

model2 = ClassificationBNNLocalRepam(config=model_config2)

# Initial parameters.
init_q_std_params2 = {
    "loc": torch.zeros(size=(model2.num_parameters,)).to(device2).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model2.num_parameters,)).to(device2) 
    * client_config2["init_var"] ** 0.5,
}

prior_std_params2 = {
    "loc": torch.zeros(size=(model2.num_parameters,)).to(device2),
    "scale": model_config2["prior_var"] ** 0.5 
    * torch.ones(size=(model2.num_parameters,)).to(device2),
}

init_factor_nat_params2 = {
    "np1": torch.zeros(model2.num_parameters).to(device2),
    "np2": torch.zeros(model2.num_parameters).to(device2),
}

p2 = MeanFieldGaussianDistribution(
    std_params=prior_std_params2, is_trainable=False
)
init_q2 = MeanFieldGaussianDistribution(
    std_params=init_q_std_params2, is_trainable=False
)

clients2 = []
size_training = 0
for i in range(100):
    data_i = client_data[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params2)
    clients2.append(
        Client(
            data=data_i,
            model=model2,
            t=t_i,
            config=client_config2,
            val_data=test_data
        )
    )    
server2 = SequentialServer(model=model2, p=p2, clients=clients2, config=server_config2, init_q=init_q2, data=train_data, val_data=test_data)

In [18]:
i = 0
while not server2.should_stop():
    server2.tick()

    # Obtain performance metrics.
    metrics2 = server2.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics2["val_mll"], metrics2["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics2["train_mll"], metrics2["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Iterations: 0.
Time taken: 409.521.
Test mll: -0.928. Test acc: 0.719.
Train mll: -0.421. Train acc: 0.874.



# Heterogeneously Split data

In [21]:
device = server_config["device"]

model = ClassificationBNNLocalRepam(config=model_config)

# Initial parameters.
init_q_std_params = {
    "loc": torch.zeros(size=(model.num_parameters,)).to(device).uniform_(-0.1, 0.1),
    "scale": torch.ones(size=(model.num_parameters,)).to(device) 
    * client_config["init_var"] ** 0.5,
}

prior_std_params = {
    "loc": torch.zeros(size=(model.num_parameters,)).to(device),
    "scale": model_config["prior_var"] ** 0.5 
    * torch.ones(size=(model.num_parameters,)).to(device),
}

init_factor_nat_params = {
    "np1": torch.zeros(model.num_parameters).to(device),
    "np2": torch.zeros(model.num_parameters).to(device),
}

p = MeanFieldGaussianDistribution(
    std_params=prior_std_params, is_trainable=False
)
init_q = MeanFieldGaussianDistribution(
    std_params=init_q_std_params, is_trainable=False
)

clients = []
size_training = 0
for i in range(100):
    data_i = client_data_2[i]
    size_training += len(data_i["x"])
    t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params)
    clients.append(
        Client(
            data=data_i,
            model=model,
            t=t_i,
            config=client_config,
            val_data=test_data
        )
    )    
server = SequentialServer(model=model, p=p, clients=clients, config=server_config, init_q=init_q, data=train_data, val_data=test_data)

In [22]:
i = 0
while not server.should_stop():
    server.tick()

    # Obtain performance metrics.
    metrics = server.log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1

0it [00:00, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Iterations: 0.
Time taken: 3093.576.
Test mll: -320.053. Test acc: 0.100.
Train mll: -328.665. Train acc: 0.100.



## Learning Rate Comparison

In [9]:
model_config = {
    "input_dim": 784,
    "latent_dim": 200,
    "output_dim": 10,
    "num_layers": 1,
    "num_predictive_samples": 100,
    "prior_var": 1.0,
}

client_config = {
    "damping_factor": 1.0,
    "optimiser": "Adam",
    "optimiser_params": {"lr": 2e-4},
    "sigma_optimiser_params": {"lr": 2e-4},
    "early_stopping": EarlyStopping(20, score_name="elbo", stash_model=True),
    "performance_metrics": performance_metrics,
    "batch_size": 512,
    "epochs": 4000,
    "print_epochs": np.inf,
    "num_elbo_samples": 10,
    "valid_factors": False,
    "device": "cpu",
    "init_var": 1e-3,
    "verbose": True,
}

learning_rates = [5e-5, 1e-4, 5e-4, 1e-3, 5e-3]

In [12]:
servers = []

for lr in learning_rates:

    client_config.update({"optimiser_params": {"lr": lr}, 
                          "sigma_optimiser_params": {"lr": lr}})

    server_config = {
        **client_config,
        #100
        "max_iterations": 15,
    }
    
    device = server_config["device"]

    model = ClassificationBNNLocalRepam(config=model_config)

    # Initial parameters.
    init_q_std_params = {
        "loc": torch.zeros(size=(model.num_parameters,)).to(device).uniform_(-0.1, 0.1),
        "scale": torch.ones(size=(model.num_parameters,)).to(device) 
        * client_config["init_var"] ** 0.5,
    }

    prior_std_params = {
        "loc": torch.zeros(size=(model.num_parameters,)).to(device),
        "scale": model_config["prior_var"] ** 0.5 
        * torch.ones(size=(model.num_parameters,)).to(device),
    }

    init_factor_nat_params = {
        "np1": torch.zeros(model.num_parameters).to(device),
        "np2": torch.zeros(model.num_parameters).to(device),
    }

    p = MeanFieldGaussianDistribution(
        std_params=prior_std_params, is_trainable=False
    )
    init_q = MeanFieldGaussianDistribution(
        std_params=init_q_std_params, is_trainable=False
    )

    clients = []
    size_training = 0
    for i in range(100):
        data_i = client_data[i]
        size_training += len(data_i["x"])
        t_i = MeanFieldGaussianFactor(nat_params=init_factor_nat_params)
        clients.append(
            Client(
                data=data_i,
                model=model,
                t=t_i,
                config=client_config,
                val_data=test_data
            )
        )

    servers.append(
        SequentialServer(
            model=model,
            p=p,
            clients=clients,
            config=server_config,
            init_q=init_q,
            data=train_data,
            val_data=test_data
        )
    )

In [13]:
j = 0

In [14]:
print("###################################")
print(f"Server: {j}")
print("###################################")

i = 0
while not servers[j].should_stop():
    servers[j].tick()

    # Obtain performance metrics.
    metrics = servers[j].log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1
j = j+1

###################################
Server: 0
###################################


0it [00:00, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Iterations: 14.
Time taken: 5951.293.
Test mll: -0.482. Test acc: 0.868.
Train mll: -0.133. Train acc: 0.967.



In [15]:
print("###################################")
print(f"Server: {j}")
print("###################################")

i = 0
while not servers[j].should_stop():
    servers[j].tick()

    # Obtain performance metrics.
    metrics = servers[j].log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1
    rng = np.random.default_rng(seed=16071)
    rng.shuffle(servers[j].clients)
j = j+1

###################################
Server: 1
###################################


0it [00:00, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Iterations: 14.
Time taken: 4884.177.
Test mll: -0.437. Test acc: 0.879.
Train mll: -0.161. Train acc: 0.965.



In [16]:
print("###################################")
print(f"Server: {j}")
print("###################################")

i = 0
while not servers[j].should_stop():
    servers[j].tick()

    # Obtain performance metrics.
    metrics = servers[j].log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1
    rng = np.random.default_rng(seed=16071)
    rng.shuffle(servers[j].clients)
j = j+1

###################################
Server: 2
###################################


0it [00:00, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Iterations: 14.
Time taken: 3154.745.
Test mll: -0.417. Test acc: 0.881.
Train mll: -0.146. Train acc: 0.967.



In [17]:
print("###################################")
print(f"Server: {j}")
print("###################################")

i = 0
while not servers[j].should_stop():
    servers[j].tick()

    # Obtain performance metrics.
    metrics = servers[j].log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1
    rng = np.random.default_rng(seed=16071)
    rng.shuffle(servers[j].clients)
j = j+1

###################################
Server: 3
###################################


0it [00:00, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Iterations: 14.
Time taken: 2827.990.
Test mll: -0.541. Test acc: 0.835.
Train mll: -0.202. Train acc: 0.945.



In [18]:
print("###################################")
print(f"Server: {j}")
print("###################################")

i = 0
while not servers[j].should_stop():
    servers[j].tick()

    # Obtain performance metrics.
    metrics = servers[j].log["performance_metrics"][-1]
    print("Iterations: {}.".format(i))
    print("Time taken: {:.3f}.".format(metrics["time"]))
    print(
      "Test mll: {:.3f}. Test acc: {:.3f}.".format(
          metrics["val_mll"], metrics["val_acc"]
      )
    )
    print(
      "Train mll: {:.3f}. Train acc: {:.3f}.\n".format(
          metrics["train_mll"], metrics["train_acc"]
      )
    )
    i += 1
    rng = np.random.default_rng(seed=16071)
    rng.shuffle(servers[j].clients)
j = j+1

###################################
Server: 4
###################################


0it [00:00, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4000 [00:00<?, ?it/s]

Iterations: 14.
Time taken: 8329.317.
Test mll: -0.526. Test acc: 0.841.
Train mll: -0.188. Train acc: 0.949.



In [31]:
plt.figure(figsize=(6.01,4))

for server, lr in zip(servers, learning_rates):
    train_acc = []
    val_acc = []
    times = []
    train_acc.append(100.0)
    val_acc.append(100.0)
    times.append(1.0)
    for metrics in server.log["performance_metrics"]:
        train_acc.append(100-metrics["train_acc"]*100)
        val_acc.append(100-metrics["val_acc"]*100)
        times.append(metrics["time"])
    plt.xscale("log")
    plt.yscale("log")
    plt.plot(times, val_acc, label=f"lr={lr}", linewidth=1)

plt.grid(visible=True)
plt.xlabel("Time")
plt.ylabel("Testing Error %")
plt.legend(loc="upper right")
plt.savefig("Learning rates on KMNIST.pgf", pad_inches=0)
plt.show()

/var/folders/6x/7wzwn4g54n38_8rfbfx611n00000gn/T/ipykernel_57962/4154917649.py:23: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
